In [ ]:
import pandas as pd

data = pd.read_excel('X Forecasting.xlsx')

In [ ]:
agg_data = data.groupby(['Calendar Day','Fiscal Year', 'Fiscal Week', 'Fiscal Year Week', 'Article: MH6 Sub Product Group']).agg({
    'Adjusted Sales': 'sum',
    'Month': lambda x: x.mode()[0]  # Use mode to select the most common month if there's any tie within the week
}).reset_index()

In [ ]:
# External Variables for Black Friday, Easter and Christmas Weeks

# Mark Black Friday weeks
black_friday_weeks = {2022: 43, 2023: 42}
agg_data['Black_Friday'] = agg_data.apply(lambda row: 1 if row['Fiscal Week'] == black_friday_weeks[row['Fiscal Year']] else 0, axis=1)
# Define the Easter weeks for both years
easter_weeks = {
    2022: [9, 10],
    2023: [10, 11]
}

# Create a new column 'Easter_Week' in agg_data and set it to 0
agg_data['Easter_Week'] = 0

# Mark the Easter weeks as 1
for year, weeks in easter_weeks.items():
    for week in weeks:
        agg_data.loc[(agg_data['Fiscal Year'] == year) & (agg_data['Fiscal Week'].isin(weeks)), 'Easter_Week'] = 1
        
#Mark Christmas Weeks as fiscal week 47 and 48
christmas_weeks= {2022: [48,49], 2023: [48,49]}
agg_data['Christmas_Week'] = 0
for year, weeks in christmas_weeks.items():
    for week in weeks:
        agg_data.loc[(agg_data['Fiscal Year'] == year) & (agg_data['Fiscal Week'].isin(weeks)), 'Christmas_Week'] = 1
        

# Statistical Models

In [ ]:
import statsmodels.api as sm

# Add a constant term for the intercept to the model
X = sm.add_constant(agg_data[['Black_Friday', 'Easter_Week', 'Month', 'Christmas_Week']])  

# Response variable
y = agg_data['Adjusted Sales']

# Fit OLS model
model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
agg_data.set_index('Calendar Day', inplace=True)
agg_data.sort_index(inplace=True)

## SARIMAX

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Iterate over each unique sub-product group
for group in agg_data['Article: MH6 Sub Product Group'].unique():
    # Filter data for the current subgroup
    subgroup_data = agg_data[agg_data['Article: MH6 Sub Product Group'] == group]

    # Define the sales data and fiscal year week for labeling
    y = subgroup_data['Adjusted Sales']
    fiscal_year_week = subgroup_data['Fiscal Year Week']

    # Plot the time series data
    plt.figure(figsize=(12, 6))
    plt.plot(y.index, y)
    plt.title(f'Time Series Plot for {group}')
    plt.xlabel('Fiscal Year Week')
    plt.ylabel('Adjusted Sales')
    plt.xticks(ticks=y.index[::2], labels=fiscal_year_week[::2], rotation=90)  # Adjust the tick frequency and rotation as needed
    plt.show()

    # Plot ACF
    plot_acf(y, lags=50)
    plt.title(f'ACF Plot for {group}')
    plt.xlabel('Lags')
    plt.ylabel('Autocorrelation')
    plt.show()

    # Plot PACF
    plot_pacf(y, lags=50)
    plt.title(f'PACF Plot for {group}')
    plt.xlabel('Lags')
    plt.ylabel('Partial Autocorrelation')
    plt.show()

# Reset index after plotting
agg_data.reset_index(inplace=True)


In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss
from arch.unitroot import PhillipsPerron

sub_product_groups = agg_data['Article: MH6 Sub Product Group'].unique()

for group in sub_product_groups:
    group_df = agg_data[agg_data['Article: MH6 Sub Product Group'] == group]
    
    # Assuming 'Adjusted Sales' is the column with the sales data
    sales = group_df['Adjusted Sales']
    
    # Perform ADF test
    adf_result = adfuller(sales.dropna())  # dropna() is used to remove any NaN values
    print(f'ADF Statistic for group {group}: {adf_result[0]}')
    print(f'p-value: {adf_result[1]}')
    
    # Perform KPSS test
    kpss_result = kpss(sales.dropna(), regression='c', nlags='auto')
    print(f'KPSS Statistic for group {group}: {kpss_result[0]}')
    print(f'p-value: {kpss_result[1]}')
    
    # Perform PP test
    pp = PhillipsPerron(sales.dropna())
    print(f'PP Statistic for group {group}: {pp.stat}')
    print(f'p-value: {pp.pvalue}')




In [ ]:

import pandas as pd
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Create a dictionary to store results
sarima_parameters = {}
model_summaries = {}
forecast_results = {}
accuracy_metrics = {}
forecast_df = pd.DataFrame()
accuracy_df = pd.DataFrame(columns=['Subproduct Group', 'MAE', 'RMSE', 'MAPE', 'wMAPE'])

# Assuming agg_data is predefined
for group in agg_data['Article: MH6 Sub Product Group'].unique():
    print(f"Running auto_arima for subgroup: {group}")

    # Prepare the data
    subgroup_data = agg_data[agg_data['Article: MH6 Sub Product Group'] == group]
    subgroup_data = subgroup_data.sort_index()
    y = subgroup_data['Adjusted Sales']
    exog = subgroup_data[['Black_Friday', 'Easter_Week', 'Christmas_Week', 'Month']]
    
    try:
        # Auto ARIMA to determine optimal parameters including seasonality
        sarimax_model = auto_arima(y, exogenous=exog, start_p=0, start_q=0, max_p=4, max_q=4,
                                   seasonal=True, m=52, trace=True, error_action='ignore', 
                                   suppress_warnings=True, stepwise=True, information_criterion='aic')
    except ValueError as e:
        if 'zero-size array to reduction operation maximum which has no identity' in str(e):
            print(f"Retrying for subgroup {group} with D=1 due to encountered error.")
            sarimax_model = auto_arima(y, exogenous=exog,
                                       start_p=0, start_q=0, max_p=4, max_q=4,
                                       start_P=0, start_Q=0, max_P=4, max_Q=4, m=52,
                                       seasonal=True, trace=True, error_action='ignore',
                                       D=1, d=None,  # Manually set seasonal differencing
                                       suppress_warnings=True, stepwise=True,  
                                       information_criterion='aic')
        else:
            raise e 

    # Store optimal parameters
    sarima_parameters[group] = (sarimax_model.order, sarimax_model.seasonal_order)
    print(f"Optimal parameters for {group}: {sarima_parameters[group]}")

    split_point = int(len(subgroup_data) * 0.8)
    train = subgroup_data.iloc[:split_point]
    test = subgroup_data.iloc[split_point:]
    exog_train = train[['Black_Friday', 'Easter_Week', 'Christmas_Week', 'Month']]
    exog_test = test[['Black_Friday', 'Easter_Week', 'Christmas_Week', 'Month']]

    # Fit SARIMAX model
    order, seasonal_order = sarima_parameters[group]
    model = SARIMAX(train['Adjusted Sales'], exog=exog_train, order=order, 
                    seasonal_order=seasonal_order, enforce_stationarity=False, 
                    enforce_invertibility=False)
    results = model.fit(disp=False)
    model_summaries[group] = results.summary()

    # Forecasting
    forecast = results.get_forecast(steps=len(test), exog=exog_test)
    forecast_mean = forecast.predicted_mean
    forecast_conf_int = forecast.conf_int()
    forecast_results[group] = {'mean': forecast_mean, 'conf_int': forecast_conf_int}

    # Calculate accuracy metrics
    errors = forecast_mean - test['Adjusted Sales']
    mae = mean_absolute_error(test['Adjusted Sales'], forecast_mean)
    rmse = np.sqrt(mean_squared_error(test['Adjusted Sales'], forecast_mean))
    mape = np.mean(np.abs(errors / test['Adjusted Sales'])) * 100
    wmape = np.sum(np.abs(errors)) / np.sum(test['Adjusted Sales']) * 100
    accuracy_metrics[group] = {'MAE': mae, 'RMSE': rmse, 'MAPE': mape, 'wMAPE': wmape}

    # Append to DataFrames
    accuracy_df = pd.concat([accuracy_df, pd.DataFrame({'Subproduct Group': [group], 'MAE': [mae], 'RMSE': [rmse], 'MAPE': [mape], 'wMAPE': [wmape]})])
    group_forecast_df = pd.DataFrame({
        'Fiscal Year Week': test.index,
        'Actual': test['Adjusted Sales'].values,
        'Forecast': forecast_mean.values,
        'Subproduct Group': group
    })
    forecast_df = pd.concat([forecast_df, group_forecast_df])

    # Plotting results
    plt.figure(figsize=(12, 6))
    plt.plot(train.index, train['Adjusted Sales'], label='Training Data')
    plt.plot(test.index, test['Adjusted Sales'], label='Actual Data')
    plt.plot(test.index, forecast_mean, label='Forecast', color='red', linestyle='--')
    plt.fill_between(test.index, forecast_conf_int.iloc[:, 0], forecast_conf_int.iloc[:, 1], color='pink', alpha=0.3)
    plt.title(f'SARIMAX Forecast vs Actuals for {group}')
    plt.legend()
    plt.show()

    print(f"Accuracy metrics for {group}: {accuracy_metrics[group]}")
    print(f"Model summary for {group}:\n{model_summaries[group]}")

print("Modeling and evaluation completed for all subproduct groups.")


In [ ]:
import matplotlib.ticker as ticker

# Additional plotting for all subproduct groups using forecast_df
for group in forecast_df['Subproduct Group'].unique():
    group_data = forecast_df[forecast_df['Subproduct Group'] == group]
    plt.figure(figsize=(12, 6))
    plt.plot(group_data['Fiscal Year Week'], group_data['Actual'], label='Actual Data')
    plt.plot(group_data['Fiscal Year Week'], group_data['Forecast'], label='Forecast', color='red', linestyle='--')
    plt.title(f'Actual vs Forecast for {group}')
    plt.xlabel('Fiscal Year Week')
    plt.ylabel('Sales')
    # Set x-ticks to the exact 'Fiscal Year Week' values with a step size of 1 and rotation of 90 degrees
    start = group_data['Fiscal Year Week'].min()
    end = group_data['Fiscal Year Week'].max()
    plt.xticks(np.arange(start, end, 1), rotation=90)
    
    # Format the x-tick labels with a precision of 4 decimal places
    formatter = ticker.FuncFormatter(lambda x, pos: '{:.4f}'.format(x))
    plt.gca().xaxis.set_major_formatter(formatter)
    
    plt.legend()
    plt.show()

# Holt's Winter

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Define the sub product groups
sub_product_groups = agg_data['Article: MH6 Sub Product Group'].unique()

# Create a plot for each sub product group
for sub_product_group in sub_product_groups:
    # Filter the data for the sub product group
    sub_data = agg_data[agg_data['Article: MH6 Sub Product Group'] == sub_product_group]['Adjusted Sales']
    
    # Perform seasonal decomposition
    result = seasonal_decompose(sub_data, period=52)  # Assuming 53 weeks in a year
    
    # Plot the decomposition
    fig, axes = plt.subplots(4, 1, figsize=(8, 6), sharex=True)
    result.observed.plot(ax=axes[0], title='Observed')
    result.trend.plot(ax=axes[1], title='Trend')
    result.seasonal.plot(ax=axes[2], title='Seasonal')
    result.resid.plot(ax=axes[3], title='Residual')
    
    # Adjust layout and add a super title
    fig.tight_layout()
    fig.subplots_adjust(top=0.9)
    fig.suptitle(f'Seasonal Decomposition of {sub_product_group}', fontsize=12)
    
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.graphics.tsaplots import plot_acf
from itertools import product
import matplotlib.pyplot as plt

# Define the parameter grid
trend_options = [None, 'add', 'mul']
seasonal_options = ['add', 'mul']
seasonal_periods_options = [52]  

# Function to evaluate a model and return its AIC
def evaluate_model(data, trend, seasonal, seasonal_periods):
    model = ExponentialSmoothing(data, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods)
    hw_fit = model.fit(optimized=True)
    return hw_fit.aic, hw_fit

# Dictionary to store the best model parameters for each subproduct group
best_model_params = {}

# DataFrame to store the residuals for each subproduct group
residuals_df = pd.DataFrame()

# Iterate over each unique subproduct group
for group in agg_data['Article: MH6 Sub Product Group'].unique():
    print(f"Running grid search for Holt-Winters model for subgroup: {group}")

    # Filter data for the current subgroup
    subgroup_data = agg_data[agg_data['Article: MH6 Sub Product Group'] == group]
    subgroup_data.sort_index(inplace=True)

    # Extract the Adjusted Sales data
    data = subgroup_data['Adjusted Sales']

    # Initialize variables to store the best model and AIC
    best_aic = float('inf')
    best_params = None
    best_model = None

    # Perform grid search over the parameter combinations
    for trend, seasonal, seasonal_periods in product(trend_options, seasonal_options, seasonal_periods_options):
        try:
            aic, model = evaluate_model(data, trend, seasonal, seasonal_periods)
            if aic < best_aic:
                best_aic = aic
                best_params = {'trend': trend, 'seasonal': seasonal, 'seasonal_periods': seasonal_periods}
                best_model = model
        except Exception as e:
            print(f"Model fitting failed for parameters trend={trend}, seasonal={seasonal}, seasonal_periods={seasonal_periods}: {e}")
            continue

    best_model_params[group] = best_params
    print(f"Best parameters for {group}: trend={best_params['trend']}, seasonal={best_params['seasonal']}, seasonal_periods={best_params['seasonal_periods']}")

    # Add the residuals to the residuals DataFrame
    residuals_df[group] = best_model.resid

    # Plot the fitted values vs actual values
    plt.figure(figsize=(10, 6))
    plt.plot(data.index, data, label='Actual Data')
    plt.plot(data.index, best_model.fittedvalues, label='Fitted Values', color='red', linestyle='--')
    plt.title(f'Actual vs Fitted Values for {group}')
    plt.xlabel('Date')
    plt.ylabel('Sales')
    plt.legend()
    plt.show()

    # Plot residuals
    plt.figure(figsize=(10, 6))
    plt.plot(data.index, best_model.resid, label='Residuals', color='blue')
    plt.axhline(0, color='red', linestyle='--')
    plt.title(f'Residuals of the Model for {group}')
    plt.xlabel('Date')
    plt.ylabel('Residuals')
    plt.legend()
    plt.show()

    # Plot ACF of residuals
    plt.figure(figsize=(10, 6))
    plot_acf(best_model.resid, lags=40)
    plt.title(f'ACF of Residuals for {group}')
    plt.xlabel('Lags')
    plt.ylabel('Autocorrelation')
    plt.show()

# Print best parameters for each group
print("Best parameters for each subproduct group:")
print(best_model_params)


In [ ]:

# Create dictionaries to store the results
hw_model_results = {}
hw_model_summaries = {}
hw_forecast_results = {}
hw_accuracy_metrics = {}

# DataFrames to store forecast and accuracy metrics
hw_forecast_df = pd.DataFrame()
hw_accuracy_df = pd.DataFrame()

# Iterate over each unique subproduct group
for group in agg_data['Article: MH6 Sub Product Group'].unique():
    print(f"Running Holt's Winter Exponential Smoothing for subgroup: {group}")

    # Filter data for the current subgroup
    subgroup_data = agg_data[agg_data['Article: MH6 Sub Product Group'] == group]

    # Ensure the DataFrame is sorted by index
    subgroup_data.sort_index(inplace=True)

    # Calculate the split point
    split_point = int(len(subgroup_data) * 0.8)

    # Split the data into training and testing sets
    train = subgroup_data.iloc[:split_point]
    test = subgroup_data.iloc[split_point:] 

    # Get the best parameters for the current subgroup
    best_params = best_model_params[group]
    
    # Fit the model using the best parameters
    model = ExponentialSmoothing(train['Adjusted Sales'],
                                 trend=best_params['trend'],
                                 seasonal=best_params['seasonal'],
                                 seasonal_periods=best_params['seasonal_periods'])
    results = model.fit()

    # Store the model and summary
    hw_model_results[group] = results
    hw_model_summaries[group] = results.summary()

    # Forecasting
    forecast = results.forecast(steps=len(test))

    # Store the forecast
    hw_forecast_results[group] = forecast

    # Calculate the errors
    errors = forecast - test['Adjusted Sales']

    # Calculate the accuracy metrics
    mae = mean_absolute_error(test['Adjusted Sales'], forecast)
    rmse = np.sqrt(mean_squared_error(test['Adjusted Sales'], forecast))
    mape = np.mean(np.abs(errors / test['Adjusted Sales'])) * 100
    wmape = np.sum(np.abs(errors)) / np.sum(test['Adjusted Sales']) * 100

    # Store the accuracy metrics
    hw_accuracy_metrics[group] = {'MAE': mae, 'RMSE': rmse, 'MAPE': mape, 'wMAPE': wmape}

    # Append the accuracy metrics to the Holt-Winters accuracy DataFrame
    hw_accuracy_df = pd.concat([hw_accuracy_df, pd.DataFrame({'Subproduct Group': [group], 'HW_MAE': [mae], 'HW_RMSE': [rmse], 'HW_MAPE': [mape], 'HW_wMAPE': [wmape]})])

    # Create a DataFrame to store forecast and actual values for this subgroup
    hw_group_forecast_df = pd.DataFrame({
        'Fiscal Year Week': test['Fiscal Year Week'],
        'Actual': test['Adjusted Sales'],
        'Forecast': forecast,
        'Subproduct Group': group
    })

    # Append the forecast data to the Holt-Winters forecast DataFrame
    hw_forecast_df = pd.concat([hw_forecast_df, hw_group_forecast_df])
    
    # Plotting the results
    plt.figure(figsize=(12, 6))
    train['Adjusted Sales'].plot(label='Training Data')
    test['Adjusted Sales'].plot(label='Actual Data')
    forecast.plot(label='Forecast', color='red', linestyle='--')
    plt.title(f"Holt's Winter Exponential Smoothing Forecast vs Actuals for {group}")
    plt.legend()
    plt.show()

    # Print the summary and accuracy metrics
    print(f"Model summary for {group}:\n{model_summaries[group]}")
    print(f"Accuracy metrics for {group}:\n{accuracy_metrics[group]}")

In [ ]:
import matplotlib.ticker as ticker

# Additional plotting for all subproduct groups using forecast_df
for group in hw_forecast_df['Subproduct Group'].unique():
    group_data = hw_forecast_df[hw_forecast_df['Subproduct Group'] == group]
    plt.figure(figsize=(12, 6))
    plt.plot(group_data['Fiscal Year Week'], group_data['Actual'], label='Actual Data')
    plt.plot(group_data['Fiscal Year Week'], group_data['Forecast'], label='Forecast', color='red', linestyle='--')
    plt.title(f'Actual vs Forecast for {group}')
    plt.xlabel('Fiscal Year Week')
    plt.ylabel('Sales')
    # Set x-ticks to the exact 'Fiscal Year Week' values with a step size of 1 and rotation of 90 degrees
    start = group_data['Fiscal Year Week'].min()
    end = group_data['Fiscal Year Week'].max()
    plt.xticks(np.arange(start, end, 1), rotation=90)
    
    # Format the x-tick labels with a precision of 4 decimal places
    formatter = ticker.FuncFormatter(lambda x, pos: '{:.4f}'.format(x))
    plt.gca().xaxis.set_major_formatter(formatter)
    
    plt.legend()
    plt.show()

# Machine Learning

## Random Forest 

In [ ]:

from sklearn.impute import SimpleImputer
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Function definitions
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def weighted_mean_absolute_percentage_error(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true) * 100

def bias(y_true, y_pred):
    return np.mean(y_true - y_pred)

# WMAPE scorer
wmape_scorer = make_scorer(weighted_mean_absolute_percentage_error, greater_is_better=False)

# Data preparation
data = agg_data.copy()  # Ensure you have loaded your data into 'agg_data'
models = {}
rf_forecast_df = pd.DataFrame()
rf_accuracy_df = pd.DataFrame()
feature_importance_rf = pd.DataFrame()

for group in data['Article: MH6 Sub Product Group'].unique():
    print(f"Training model for subgroup: {group}")
    subgroup_data = data[data['Article: MH6 Sub Product Group'] == group]
    
    #Features for Cyclicity
    subgroup_data['sin_fiscal_week'] = np.sin(2 * np.pi * subgroup_data['Fiscal Week'] / 52)
    subgroup_data['cos_fiscal_week'] = np.cos(2 * np.pi * subgroup_data['Fiscal Week'] / 52)
    
    
    X = subgroup_data[['Fiscal Year', 'Fiscal Week', 'Month', 'Black_Friday', 'Easter_Week', 'Christmas_Week', 'sin_fiscal_week', 'cos_fiscal_week']]
    y = subgroup_data['Adjusted Sales']
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Initialize features for X_test
    for lag in range(1, 5):
        X_test[f'lag_{lag}_week'] = np.concatenate((y_train.tail(lag).values, y_test.head(len(y_test) - lag)))
    for window in [2, 4]:
        rolling_series = pd.concat([y_train.tail(window-1), y_test])
        X_test[f'Rolling_Mean_{window}'] = rolling_series.rolling(window).mean().iloc[window-1:]
        X_test[f'Rolling_Std_{window}'] = rolling_series.rolling(window).std().iloc[window-1:]

    # Cross-validation setup
    min_train_weeks = 52
    val_weeks = 8
    n_folds = (len(X_train) - min_train_weeks) // val_weeks
    tscv = TimeSeriesSplit(n_splits=n_folds, test_size=val_weeks)
    param_grid = {
        'max_depth': [None, 3, 5, 7, 10],
        'n_estimators': [50, 100, 200, 300],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    model = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring=wmape_scorer, cv=tscv, verbose=1)

    # Cross-validation
    for i, (train_index, val_index) in enumerate(tscv.split(X_train)):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        # Add features for each fold
        for lag in range(1, 5):
            X_train_fold[f'lag_{lag}_week'] = y_train_fold.shift(lag)
            X_val_fold[f'lag_{lag}_week'] = np.concatenate((y_train_fold.tail(lag).values, y_val_fold.head(len(y_val_fold) - lag)))
        for window in [2, 4]:
            X_train_fold[f'Rolling_Mean_{window}'] = y_train_fold.rolling(window=window).mean()
            X_train_fold[f'Rolling_Std_{window}'] = y_train_fold.rolling(window=window).std()
            temp_series = pd.concat([y_train_fold.tail(window-1), y_val_fold])
            X_val_fold[f'Rolling_Mean_{window}'] = temp_series.rolling(window=window).mean().iloc[window-1:]
            X_val_fold[f'Rolling_Std_{window}'] = temp_series.rolling(window=window).std().iloc[window-1:]
            
        # Impute strategy: mean, median, most_frequent, or constant (fill with a specific value)
        imputer = SimpleImputer(strategy='mean')  # or 'median', 'most_frequent'

        # Impute missing values
        X_train_fold_imputed = pd.DataFrame(imputer.fit_transform(X_train_fold), columns=X_train_fold.columns)
        X_val_fold_imputed = pd.DataFrame(imputer.transform(X_val_fold), columns=X_val_fold.columns)

        grid_search.fit(X_train_fold_imputed, y_train_fold)
        y_pred_val = grid_search.predict(X_val_fold_imputed)
        wmape_val = weighted_mean_absolute_percentage_error(y_val_fold, y_pred_val)
        print(f"Fold {i+1} - Validation wMAPE: {wmape_val}")

    # Best model and predictions
    best_model = grid_search.best_estimator_
    models[group] = best_model
    print(f"Best hyperparameters for {group}: {grid_search.best_params_}")
    
    # Print feature importances
    feature_importances = best_model.feature_importances_
    print(f"Feature importances for {group}:")
    for feature, importance in zip(X_train_fold_imputed.columns, feature_importances):
        print(f"{feature}: {importance}")
    
    
    y_pred = best_model.predict(X_test)
    
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    wmape = weighted_mean_absolute_percentage_error(y_test, y_pred)
    bias_val = bias(y_test, y_pred)
    print(f"Test Set Evaluation for {group}:")
    print(f"Mean Squared Error: {mse}")
    print(f"Root Mean Squared Error: {rmse}")
    print(f"Mean Absolute Percentage Error: {mape}")
    print(f"Weighted Mean Absolute Percentage Error: {wmape}")
    print(f"Bias: {bias_val}")

    # Append metrics and forecast
    rf_accuracy_df = pd.concat([rf_accuracy_df, pd.DataFrame({'Subproduct Group': [group], 'RF_MAPE': [mape], 'RF_RMSE': [rmse], 'RF_wMAPE': [wmape]})])
    rf_group_forecast_df = pd.DataFrame({
        'Fiscal Year Week': X_test.index,
        'Actual': y_test.values,
        'Forecast': y_pred,
        'Subproduct Group': group
    })
    rf_forecast_df = pd.concat([rf_forecast_df, rf_group_forecast_df])

    #Save feature importances
    imp_rf = pd.DataFrame({
                'Feature': X_train_fold_imputed.columns,
                'Importance': best_model.feature_importances_,
                'Subproduct Group': group
            })
    feature_importance_rf = pd.concat([feature_importance_rf, imp_rf], ignore_index=True)
    
    X_test['Fiscal Year Week'] = X_test['Fiscal Year'].astype(str) + ' - ' + X_test['Fiscal Week'].astype(str)

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(X_test['Fiscal Year Week'], y_test.values, label='Actual')
    plt.plot(X_test['Fiscal Year Week'], y_pred, label='Predicted')
    plt.title(f"Actual vs Predicted Sales for {group}")
    plt.xlabel('Fiscal Year Week')
    plt.xticks(rotation='vertical')
    plt.ylabel('Sales')
    plt.legend()
    plt.tight_layout()
    plt.show()


### Feature Importance

In [ ]:
import seaborn as sns

# Aggregate and visualize feature importances according to ascending order for each group
for grp in feature_importance_rf['Subproduct Group'].unique():
    group_data = feature_importance_rf[feature_importance_rf['Subproduct Group'] == grp].sort_values(by='Importance', ascending=False)
    plt.figure(figsize=(8, 6))
    sns.barplot(data=group_data, x='Importance', y='Feature', ci=None)
    plt.title(f'Feature Importances for {grp}')
    plt.show()

# Overall feature importances
overall_importances = feature_importance_rf.groupby('Feature')['Importance'].mean().sort_values(ascending=False)
plt.figure(figsize=(8, 6))
sns.barplot(x=overall_importances.values, y=overall_importances.index, orient='h')
plt.title('Average Feature Importances Across All Groups')
plt.show()

## XGBoost 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Define scoring functions
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def weighted_mean_absolute_percentage_error(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true) * 100

def bias(y_true, y_pred):
    return np.mean(y_true - y_pred)

# WMAPE scorer
wmape_scorer = make_scorer(weighted_mean_absolute_percentage_error, greater_is_better=False)

# Data preparation
data = agg_data.copy()  # Assume 'agg_data' is previously loaded
models = {}
feature_importance_df = pd.DataFrame()
xgb_forecast_df = pd.DataFrame()
xgb_accuracy_df = pd.DataFrame()

for group in data['Article: MH6 Sub Product Group'].unique():
    print(f"Training model for subgroup: {group}")
    subgroup_data = data[data['Article: MH6 Sub Product Group'] == group]
    
    subgroup_data['sin_fiscal_week'] = np.sin(2 * np.pi * subgroup_data['Fiscal Week'] / 52)
    subgroup_data['cos_fiscal_week'] = np.cos(2 * np.pi * subgroup_data['Fiscal Week'] / 52)
    
    X = subgroup_data[['Fiscal Year', 'Fiscal Week', 'Month', 'Black_Friday', 'Easter_Week', 'Christmas_Week', 'sin_fiscal_week', 'cos_fiscal_week']]
    y = subgroup_data['Adjusted Sales']
    
    #Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Initialize features for X_test
    feature_names = ['lag_1_week', 'lag_2_week', 'lag_3_week', 'lag_4_week', 'Rolling_Mean_2', 'Rolling_Mean_4', 'Rolling_Std_2', 'Rolling_Std_4']
    for lag in range(1, 5):
        X_test[f'lag_{lag}_week'] = np.concatenate((y_train.tail(lag).values, y_test.head(len(y_test) - lag)))
    
    for window in [2, 4]:
        rolling_series = pd.concat([y_train.tail(window-1), y_test])
        X_test[f'Rolling_Mean_{window}'] = rolling_series.rolling(window).mean().iloc[window-1:]
        X_test[f'Rolling_Std_{window}'] = rolling_series.rolling(window).std().iloc[window-1:]

    # Cross-validation setup
    min_train_weeks = 52
    val_weeks = 8
    n_folds = (len(X_train) - min_train_weeks) // val_weeks
    tscv = TimeSeriesSplit(n_splits=n_folds, test_size=val_weeks)
    param_grid = {
        'max_depth': [None, 3, 5, 7, 10],
        'n_estimators': [50, 100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.7, 0.8, 0.9]
    }
    model = XGBRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring=wmape_scorer, cv=tscv, verbose=1)

    # Cross-validation
    for i, (train_index, val_index) in enumerate(tscv.split(X_train)):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # Add features for each fold
        for lag in range(1, 5):
            X_train_fold[f'lag_{lag}_week'] = y_train_fold.shift(lag)
            X_val_fold[f'lag_{lag}_week'] = np.concatenate((y_train_fold.tail(lag).values, y_val_fold.head(len(y_val_fold) - lag)))
        
        for window in [2, 4]:
            X_train_fold[f'Rolling_Mean_{window}'] = y_train_fold.rolling(window=window).mean()
            X_train_fold[f'Rolling_Std_{window}'] = y_train_fold.rolling(window=window).std()
            temp_series = pd.concat([y_train_fold.tail(window-1), y_val_fold])
            X_val_fold[f'Rolling_Mean_{window}'] = temp_series.rolling(window=window).mean().iloc[window-1:]
            X_val_fold[f'Rolling_Std_{window}'] = temp_series.rolling(window=window).std().iloc[window-1:]

        grid_search.fit(X_train_fold, y_train_fold)
        y_pred_val = grid_search.predict(X_val_fold)
        wmape_val = weighted_mean_absolute_percentage_error(y_val_fold, y_pred_val)
        print(f"Fold {i+1} - Validation wMAPE: {wmape_val}")
        
        
    # Best model and predictions
    best_model = grid_search.best_estimator_
    models[group] = best_model
    print(f"Best hyperparameters for {group}: {grid_search.best_params_}")
    
    # Print feature importances
    feature_importances = best_model.feature_importances_
    print(f"Feature importances for {group}:")
    for feature, importance in zip(X_train_fold.columns, feature_importances):
        print(f"{feature}: {importance}")
    
    
    y_pred = best_model.predict(X_test)
    
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    wmape = weighted_mean_absolute_percentage_error(y_test, y_pred)
    bias_val = bias(y_test, y_pred)
    print(f"Test Set Evaluation for {group}:")
    print(f"Mean Squared Error: {mse}")
    print(f"Root Mean Squared Error: {rmse}")
    print(f"Mean Absolute Percentage Error: {mape}")
    print(f"Weighted Mean Absolute Percentage Error: {wmape}")
    print(f"Bias: {bias_val}")

    # Append metrics and forecast
    xgb_accuracy_df = pd.concat([xgb_accuracy_df, pd.DataFrame({'Subproduct Group': [group], 'XGB_MAPE': [mape], 'XGB_RMSE': [rmse], 'XGB_wMAPE': [wmape]})])
    xgb_group_forecast_df = pd.DataFrame({
            'Fiscal Year Week': X_test.index,
            'Actual': y_test.values,
            'Forecast': y_pred,
            'Subproduct Group': group
        })
    xgb_forecast_df = pd.concat([xgb_forecast_df, xgb_group_forecast_df])
        
    # Save feature importances
    imp_df = pd.DataFrame({
                'Feature': X_train_fold.columns,
                'Importance': best_model.feature_importances_,
                'Subproduct Group': group
            })
    feature_importance_df = pd.concat([feature_importance_df, imp_df], ignore_index=True)

 
    X_test['Fiscal Year Week'] = X_test['Fiscal Year'].astype(str) + ' - ' + X_test['Fiscal Week'].astype(str)
        
    # Plot actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(X_test['Fiscal Year Week'], y_test.values, label='Actual')
    plt.plot(X_test['Fiscal Year Week'], y_pred, label='Predicted')
    plt.title(f"Actual vs Predicted Sales for {group}")
    plt.xlabel('Fiscal Year Week')
    plt.xticks(rotation='vertical')
    plt.ylabel('Sales')
    plt.legend()
    plt.tight_layout()
    plt.show()



### Feature Importance

In [ ]:
 # Aggregate and visualize feature importances according to ascending order for each group
for grp in feature_importance_df['Subproduct Group'].unique():
    group_data = feature_importance_df[feature_importance_df['Subproduct Group'] == grp].sort_values(by='Importance', ascending=False)
    plt.figure(figsize=(8, 6))
    sns.barplot(data=group_data, x='Importance', y='Feature', ci=None)
    plt.title(f'Feature Importances for {grp}')
    plt.show()

# Overall feature importances
overall_importances = feature_importance_df.groupby('Feature')['Importance'].mean().sort_values(ascending=False)
plt.figure(figsize=(8, 6))
sns.barplot(x=overall_importances.values, y=overall_importances.index, orient='h')
plt.title('Average Feature Importances Across All Groups')
plt.show()